# LangChain 短期记忆

短期记忆可以让模型知道先前的交互，从而使模型可以更好的回答当前问题。

对话历史是短期记忆的最常见形式。长对话对于当前模型来说是一个挑战，因为模型无法承受较大的、完成的上下文窗口，会导致上下文丢失、错误。即使模型支持很大的上下文窗口，但是如果输入信息过多，会导致模型注意力分散，导致输出结果不佳、输出速度缓慢。

## 用法

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langgraph.checkpoint.memory import InMemorySaver
in_memory_server = InMemorySaver()

from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model="deepseek-chat")
agent = create_agent(model=llm,checkpointer=in_memory_server)
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Hi,My name is Bob"
            }
        ]
    },
    {
        "configurable": {
            "thread_id": 1
        }
    }
)
from langchain.messages import AIMessage
for message in response.get("messages"):
    if isinstance(message, AIMessage):
        print(f"第一次对话结果:{message.content}")

response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What am my name?"
            }
        ]
    },
    {
        
        "configurable": {
            "thread_id": 1
        }
    }
)
for message in response.get("messages"):
    if isinstance(message, AIMessage):
        print(f"第二次对话结果:{message.content}")

## 短期记忆超过模型输入上下文限制时集中处理方式

1. 截断消息：截断前N或者后N条消息
2. 删除消息：删除之前的所有消息
3. 总结消息：对之前历史上下文进行总结
4. 自定义策略：例如消息 过滤等方式

## 访问记忆

1. 在工具中可以读取、写入记忆。
2. 在中间件中访问短期记忆（状态），以根据对话历史或自定义状态字段创建动态提示。
3. 在@before_model中间件中访问短期记忆（状态），以在模型调用之前处理消息
4. 在@after_model中间件中访问短期记忆（状态），以在模型调用之后处理消息。